# Code


In [5]:
# CLIPModel Pipeline with SimpleViT, BatchNorm, LayerNorm, Dropout, AMP, Scheduler, Early Stopping
# This script processes genomic inputs (sequence + pattern + frequency) and trains a CLIP-like model using SimpleViT encoders

import os
import random
import math
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR

# Import SimpleViT from lucidrains implementation
from vit_pytorch.simple_vit import SimpleViT

# Optional mixed precision
from torch.amp import GradScaler, autocast

# -------------------- Flags & Hyperparameters --------------------
LOAD_DATASET    = True      # If True, load from saved .npy files instead of generating
SAVE_DATASET    = False     # If True (and generating), save arrays to .npy after generation

n_epochs        = 10000
batch_size      = 2_000
learning_rate   = 1e-2               # adjust as needed
weight_decay    = 5e-8               # weight decay for regularization
dropout_prob    = 0.0                # dropout probability
warmup_epochs   = 50                 # number of warmup epochs
USE_AMP         = False              # set to True to enable mixed precision
patience        = 1000               # early stopping patience
# -------------------------------------------------------------------

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==============================================================================
# 1. Load promoter coordinates from BED file
# ==============================================================================
promoter_bed = 'promoter_view_test.bed'  # replace with actual path
bed_df = pd.read_csv(
    promoter_bed,
    sep='\t',
    header=None,
    usecols=[0, 1, 2],
    names=['chrom', 'start', 'end']
)

# ==============================================================================
# 2. Load intervals ±25 kb of promoter start (preprocessed into numpy arrays)
# ==============================================================================
# seq_h_dataset shape: (n_samples, 5, 32) → 5 rows, 32 nucleotides per row (one-hot or embedding channels)
# top_out_dataset shape: (n_samples, 10, 100) → 10 pattern rows, 100 bp per pattern (binary patterns)
# top_amount_dataset shape: (n_samples, 10) → frequency vector per pattern
seq_h_dataset      = np.load('seq_h_dataset.npy')       # shape (n_samples, 5, 32)
top_out_dataset    = np.load('top_out_dataset.npy')     # shape (n_samples, 10, 100)
top_amount_dataset = np.load('top_amount_dataset.npy')  # shape (n_samples, 10)
n_samples          = seq_h_dataset.shape[0]

# ==============================================================================
# 3. Split into Training (80%) and Validation (20%) Sets
# ==============================================================================
indices = np.arange(n_samples)
np.random.seed(42)
np.random.shuffle(indices)
split     = int(0.8 * n_samples)
train_idx = indices[:split]
val_idx   = indices[split:]

train_seq_h      = seq_h_dataset[train_idx]
train_top_out    = top_out_dataset[train_idx]
train_top_amount = top_amount_dataset[train_idx]
val_seq_h        = seq_h_dataset[val_idx]
val_top_out      = top_out_dataset[val_idx]
val_top_amount   = top_amount_dataset[val_idx]

# ==============================================================================
# 4. Wrap in PyTorch Dataset and DataLoader
# ==============================================================================
class GenomicClipDataset(Dataset):
    def __init__(self, seq_h_array, top_array, top_amount_array):
        assert seq_h_array.shape[0] == top_array.shape[0] == top_amount_array.shape[0]
        self.seq_h      = seq_h_array
        self.top        = top_array
        self.top_amount = top_amount_array

    def __len__(self):
        return self.seq_h.shape[0]

    def __getitem__(self, idx):
        # dna: (5, 32) → convert channels dimension dynamically
        dna  = torch.from_numpy(self.seq_h[idx]).float()       # (5, 32)
        # patterns: (10, 100)
        pat  = torch.from_numpy(self.top[idx]).float()         # (10, 100)
        # frequency: (10,)
        freq = torch.from_numpy(self.top_amount[idx]).float()  # (10,)
        # Add channel dimension for ViT: dna → (channels, H, W)
        # Here, treat '5' as channels or height? We treat 5 rows as height and 1 channel, so reshape to (1,5,32)
        dna = dna.unsqueeze(0)  # → (1, 5, 32)
        # For patterns: treat 10 rows as height, 1 channel, 100 width: reshape (1,10,100)
        pat = pat.unsqueeze(0)  # → (1, 10, 100)
        return dna, pat, freq

train_loader = DataLoader(
    GenomicClipDataset(train_seq_h, train_top_out, train_top_amount),
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)
val_loader = DataLoader(
    GenomicClipDataset(val_seq_h, val_top_out, val_top_amount),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

# ==============================================================================
# 5. Define CLIPModel with SimpleViT encoders
# ==============================================================================
class CLIPModel(nn.Module):
    def __init__(self,
                 dna_dim: int = 256,
                 dna_depth: int = 2,
                 dna_heads: int = 2,
                 pat_dim: int = 2512,
                 pat_depth: int = 2,
                 pat_heads: int = 2,
                 latent_dim: int = 2512,
                 dropout: float = 0.4,
                 dna_channels: int = 100,
                 pat_channels: int = 100):
        super(CLIPModel, self).__init__()

        # --- DNA Encoder (SimpleViT) ---
        # dna_tensor shape → (B, dna_channels, 5, 32)
        self.dna_encoder = SimpleViT(
            image_size=(5, 32),        # height x width for dna
            patch_size=(5, 8),         # patch dims
            num_classes=dna_dim,
            dim=dna_dim,
            depth=dna_depth,
            heads=dna_heads,
            mlp_dim=dna_dim * 2,
            channels=dna_channels,
            dim_head=64
        )
        self.dna_proj = nn.Sequential(
            nn.Linear(dna_dim, latent_dim),
            nn.LayerNorm(latent_dim),     # retain LayerNorm here
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
        )

        # --- Pattern Encoder (SimpleViT) ---
        # pat_tensor shape → (B, pat_channels, 10, 100)
        self.pat_encoder = SimpleViT(
            image_size=(10, 100),
            patch_size=(2, 10),
            num_classes=pat_dim,
            dim=pat_dim,
            depth=pat_depth,
            heads=pat_heads,
            mlp_dim=pat_dim * 2,
            channels=pat_channels,
            dim_head=64
        )
        self.pat_feat_bn = nn.BatchNorm1d(pat_dim, eps=1e-5, momentum=0.1)

        # --- Frequency branch normalization ---
        self.freq_bn = nn.BatchNorm1d(10, eps=1e-5, momentum=0.1)
        self.freq_proj_input = nn.Linear(10, pat_dim)
        self.freq_mlp = nn.Sequential(
            nn.Linear(pat_dim, pat_dim),
            nn.LayerNorm(pat_dim),       # keep LayerNorm inside freq MLP
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(pat_dim, pat_dim),
            nn.LayerNorm(pat_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
        )

        # --- Combiner MLP: (pat_dim*2 → pat_dim) with BatchNorm after first linear
        self.combiner_mlp = nn.Sequential(
            nn.Linear(pat_dim * 2, pat_dim),
            nn.BatchNorm1d(pat_dim, eps=1e-5, momentum=0.1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(pat_dim, pat_dim),
            nn.LayerNorm(pat_dim),       # final LayerNorm
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
        )

        # Final projection: (pat_dim → latent_dim)
        self.pat_proj = nn.Sequential(
            nn.Linear(pat_dim, latent_dim),
            nn.LayerNorm(latent_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
        )

        # Learnable temperature parameter (initialized to log(1/0.07))
        self.logit_scale_param = nn.Parameter(torch.ones([]) * math.log(1/0.07))

        # Xavier initialization for all Linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, dna_tensor, pat_tensor, freq_tensor):
        """
        dna_tensor:  (B, channels, 5, 32)
        pat_tensor:  (B, channels, 10, 100)
        freq_tensor: (B, 10)

        Returns:
            dna_latent: (B, latent_dim)
            pat_latent: (B, latent_dim)
            logit_scale: scalar > 0
        """
        # --- DNA branch ---
        dna_feat   = self.dna_encoder(dna_tensor)      # (B, dna_dim)
        dna_latent = self.dna_proj(dna_feat)           # (B, latent_dim)

        # --- Pattern branch ---
        pat_feat       = self.pat_encoder(pat_tensor)  # (B, pat_dim)
        pat_feat_norm  = self.pat_feat_bn(pat_feat)    # (B, pat_dim)

        # --- Frequency branch ---
        freq_normed = self.freq_bn(freq_tensor)       # (B, 10)
        freq_proj   = self.freq_proj_input(freq_normed)  # (B, pat_dim)
        freq_feat   = self.freq_mlp(freq_proj)           # (B, pat_dim)

        # --- Combine pattern + frequency ---
        combined   = torch.cat([pat_feat_norm, freq_feat], dim=1)  # (B, 2*pat_dim)
        comb_feat  = self.combiner_mlp(combined)                   # (B, pat_dim)
        pat_latent = self.pat_proj(comb_feat)                       # (B, latent_dim)

        # Clamp logit_scale to avoid numerical explosion
        logit_scale = torch.clamp(self.logit_scale_param, -5.0, 5.0).exp()
        return dna_latent, pat_latent, logit_scale

# Instantiate model and move to device (example dynamic channels)
# dna_channels = number of input channels for DNA (e.g., 1 if one-hot encoded)
# pat_channels = number of input channels for patterns (e.g., 1 for binary patterns)
model = CLIPModel(
    dna_dim=256, dna_depth=8, dna_heads=8,
    pat_dim=512, pat_depth=8, pat_heads=8,
    latent_dim=32, dropout=dropout_prob,
    dna_channels=1, pat_channels=1
).to(device)

# ==============================================================================
# 6. Contrastive Loss (InfoNCE with Label Smoothing)
# ==============================================================================
def contrastive_loss(dna_latent, pat_latent, logit_scale, smoothing: float = 0.1):
    """
    dna_latent:   (B, D)
    pat_latent:   (B, D)
    logit_scale:  scalar
    smoothing:    float in [0,1]
    Returns:
        average of image-to-text and text-to-image KLDiv-based losses.
    """
    B, D = dna_latent.size()
    eps = 1e-8
    dna_norm = dna_latent / (dna_latent.norm(dim=1, keepdim=True) + eps)
    pat_norm = pat_latent / (pat_latent.norm(dim=1, keepdim=True) + eps)
    logits = logit_scale * torch.matmul(dna_norm, pat_norm.t())  # (B, B)
    labels = torch.arange(B, device=logits.device)
    n_classes = logits.size(1)
    with torch.no_grad():
        smooth_target = torch.full((B, n_classes), smoothing / (n_classes - 1), device=logits.device)
        smooth_target[torch.arange(B), labels] = 1.0 - smoothing
    loss_i2p = F.kl_div(F.log_softmax(logits, dim=1), smooth_target, reduction="batchmean")
    loss_p2i = F.kl_div(F.log_softmax(logits.t(), dim=1), smooth_target, reduction="batchmean")
    return (loss_i2p + loss_p2i) / 2.0

# ==============================================================================
# 7. Training & Validation Loop with Optional AMP, Extended Warmup, Early Stopping
# ==============================================================================
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps  = n_epochs * len(train_loader)
warmup_steps = warmup_epochs * len(train_loader)

def lr_lambda(step):
    if step < warmup_steps:
        return float(step) / float(max(1, warmup_steps))
    return 0.5 * (1.0 + math.cos(math.pi * (step - warmup_steps) / (total_steps - warmup_steps)))

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)
scaler = GradScaler(enabled=USE_AMP)

# Debug: frequency stats
freq_all = torch.from_numpy(top_amount_dataset)
print(
    "Initial freq stats:",
    f"min={freq_all.min().item():.1e}, max={freq_all.max().item():.1e},",
    f"mean={freq_all.mean().item():.1e}, std={freq_all.std().item():.1e}"
)

best_val_loss = float('inf')
no_improve    = 0
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(1, n_epochs + 1):
    # --- Training Phase ---
    model.train()
    train_loss_accum = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{n_epochs} [Train]", leave=False)

    for dna_batch, pat_batch, freq_batch in train_bar:
        dna_batch, pat_batch, freq_batch = dna_batch.to(device), pat_batch.to(device), freq_batch.to(device)
        if dna_batch.isnan().any() or dna_batch.isinf().any():
            raise RuntimeError("dna_batch contains NaN or Inf")
        if pat_batch.isnan().any() or pat_batch.isinf().any():
            raise RuntimeError("pat_batch contains NaN or Inf")
        if freq_batch.isnan().any() or freq_batch.isinf().any():
            raise RuntimeError("freq_batch contains NaN or Inf")

        optimizer.zero_grad()
        if USE_AMP:
            with autocast():
                dna_latent, pat_latent, logit_scale = model(dna_batch, pat_batch, freq_batch)
                loss = contrastive_loss(dna_latent, pat_latent, logit_scale)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
        else:
            dna_latent, pat_latent, logit_scale = model(dna_batch, pat_batch, freq_batch)
            loss = contrastive_loss(dna_latent, pat_latent, logit_scale)
            loss.backward()

        # Gradient clipping & check
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        for name, param in model.named_parameters():
            if param.grad is not None and param.grad.isnan().any():
                raise RuntimeError(f"Gradient for {name} contains NaN")

        if USE_AMP:
            scaler.step(optimizer)
            scaler.update()
        else:
            optimizer.step()
        scheduler.step()

        train_loss_accum += loss.item()
        train_bar.set_postfix(train_loss=f"{loss.item():.4f}")
    train_bar.close()

    avg_train_loss = train_loss_accum / len(train_loader)
    model.eval()
    val_loss_accum = 0.0
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch}/{n_epochs} [Val]", leave=False)

    with torch.no_grad():
        for dna_batch, pat_batch, freq_batch in val_bar:
            dna_batch, pat_batch, freq_batch = dna_batch.to(device), pat_batch.to(device), freq_batch.to(device)
            if USE_AMP:
                with autocast():
                    dna_latent, pat_latent, logit_scale = model(dna_batch, pat_batch, freq_batch)
                    loss = contrastive_loss(dna_latent, pat_latent, logit_scale)
            else:
                dna_latent, pat_latent, logit_scale = model(dna_batch, pat_batch, freq_batch)
                loss = contrastive_loss(dna_latent, pat_latent, logit_scale)
            val_loss_accum += loss.item()
            val_bar.set_postfix(val_loss=f"{loss.item():.4f}")
    val_bar.close()

    avg_val_loss = val_loss_accum / len(val_loader)
    print(
        f"Epoch {epoch}/{n_epochs} — "
        f"Avg Train Loss: {avg_train_loss:.4f} | "
        f"Avg Val Loss:   {avg_val_loss:.4f} | "
        f"LR: {scheduler.get_last_lr()[0]:.2e}"
    )

    # Early Stopping & Checkpoint
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve    = 0
        ckpt_path     = os.path.join(checkpoint_dir, "best_clip_model.pth")
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "scheduler_state": scheduler.state_dict(),
            "best_val_loss": best_val_loss
        }, ckpt_path)
        print(f"  ► Saved new best checkpoint (Val Loss: {best_val_loss:.4f})")
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"No improvement for {patience} epochs. Stopping early at epoch {epoch}.")
            break

# ==============================================================================
# 8. Save Final Model Checkpoint (if not already saved in early stop)
# ==============================================================================
final_ckpt = os.path.join(checkpoint_dir, "last_epoch_clip_model.pth")
torch.save({
    "epoch": epoch,
    "model_state": model.state_dict(),
    "optimizer_state": optimizer.state_dict(),
    "scheduler_state": scheduler.state_dict(),
    "best_val_loss": best_val_loss
}, final_ckpt)
print(f"Training complete. Final checkpoint saved to {final_ckpt}")

Initial freq stats: min=0.0e+00, max=4.2e+03, mean=3.2e+02, std=7.7e+02


Epoch 1/10000 — Avg Train Loss: 6.6616 | Avg Val Loss:   6.5027 | LR: 2.00e-04
  ► Saved new best checkpoint (Val Loss: 6.5027)


Epoch 2/10000 — Avg Train Loss: 6.4817 | Avg Val Loss:   6.5026 | LR: 4.00e-04
  ► Saved new best checkpoint (Val Loss: 6.5026)


Epoch 3/10000 — Avg Train Loss: 6.4747 | Avg Val Loss:   6.5187 | LR: 6.00e-04


Epoch 4/10000 — Avg Train Loss: 6.4710 | Avg Val Loss:   6.5013 | LR: 8.00e-04
  ► Saved new best checkpoint (Val Loss: 6.5013)


...

Epoch 975/10000 — Avg Train Loss: 3.9938 | Avg Val Loss:   11.7342 | LR: 9.79e-03


Epoch 976/10000 — Avg Train Loss: 3.9898 | Avg Val Loss:   10.8832 | LR: 9.79e-03


Epoch 977/10000 — Avg Train Loss: 4.0138 | Avg Val Loss:   11.3577 | LR: 9.79e-03


Epoch 978/10000 — Avg Train Loss: 4.0598 | Avg Val Loss:   10.9990 | LR: 9.79e-03


KeyboardInterrupt: 

In [ ]:
# Overfit is working
# dropout is regulating it right now


In [ ]:
#clip
first modality

DNA 2d matrix
  - 32 bp binary hotencoder ((5, 32)) #ACTGN

Second clip MODALITY

2d matrix (peak binarized top 10 encoders) + 1d vector with values (frequency of the top 10 encoders)

  - (10,100) binary sparse_matrix
  - frequency of occurency for values for each row # This information should be used to reflect the frequency


use VIT to process both modules,
add batch norm when needed
use the lucid rains implementation of x-clip and VIT


